In [ ]:
%pip install nltk transformers torch annoy seaborn matplotlib scikit-learn PyPDF2 plotly


In [14]:
import nltk
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import BertTokenizer, BertModel
import PyPDF2
import os
import pickle
from tqdm import tqdm
from huggingface_hub import snapshot_download
import os
import numpy as np
import PyPDF2
from annoy import AnnoyIndex



from collections import defaultdict
from tqdm import tqdm
from multiprocessing import Pool
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from torch.nn import DataParallel
import torch
from paper_processing_for_embeddings import preprocess_and_read 

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/aayushgupta/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aayushgupta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
# Check if CUDA is available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)

# If multiple GPUs are available, use DataParallel
if torch.cuda.device_count() > 1:
    model = DataParallel(model)

model = model.to(device)



In [16]:
def embed_words_batch(words):
    inputs = tokenizer(words, padding=True, return_tensors='pt', truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Move inputs to the appropriate device
    outputs = model(**inputs)
    return outputs.hidden_states[-1][:, 0, :].detach().cpu().numpy()


In [5]:
def build_annoy_index(embeddings_dict):
    f = list(embeddings_dict.values())[0]['embedding'].shape[0]
    t = AnnoyIndex(f, 'angular')
    for i, (word, data) in enumerate(embeddings_dict.items()):
        t.add_item(i, data['embedding'])
    t.build(10)
    return t

def query_similar_words(query, index, embeddings_dict, top_n=5):
    query_embedding = embed_words_batch([query])[0]  # Embed the query word
    nearest_ids = index.get_nns_by_vector(query_embedding, top_n)

    similar_words_with_titles = []
    for i in nearest_ids:
        word = list(embeddings_dict.keys())[i]
        title = embeddings_dict[word]['file'].split('/')[-1]  # Extract the file name
        similar_words_with_titles.append((word, title))

    return similar_words_with_titles



In [6]:
with open('word_embeddings.pkl', 'rb') as f:
    embeddings_dict = pickle.load(f)


In [7]:
# Build Annoy index
annoy_index = build_annoy_index(embeddings_dict)

In [ ]:
# Query for similar words
query_word = "Chain-of-Thought"  # Replace with your query word
similar_words = query_similar_words(query_word, annoy_index, embeddings_dict, top_n=50)
print(f"Words similar to '{query_word}': {similar_words}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.cluster import KMeans

def optimized_plot_embeddings(embeddings_dict, query, index, top_n=5):
    # Extract embeddings and corresponding words
    words, embeddings = zip(*[(word, data['embedding']) for word, data in embeddings_dict.items() if data['embedding'] is not None])

    # Randomly sample 1000 embeddings
    sample_size = min(100000, len(embeddings))  # Adjust sample size as needed
    sampled_indices = random.sample(range(len(embeddings)), sample_size)
    sampled_embeddings = [embeddings[i] for i in sampled_indices]
    sampled_words = [words[i] for i in sampled_indices]

    # Add the query embedding
    query_embedding = embeddings_dict.get(query, {'embedding': None})['embedding']
    if query_embedding is None:
        query_embedding = embed_words_batch([query])[0]  # Replace with your actual embedding function

    # Add nearest neighbors of the query
    nearest_neighbors_indices = index.get_nns_by_vector(query_embedding, top_n)
    nearest_neighbors_embeddings = [embeddings[i] for i in nearest_neighbors_indices]
    nearest_neighbors_words = [words[i] for i in nearest_neighbors_indices]

    # Combine sampled embeddings, query, and nearest neighbors
    extended_embeddings = np.vstack(sampled_embeddings + [query_embedding] + nearest_neighbors_embeddings)

    # TSNE for dimensionality reduction
    tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
    tsne_results = tsne.fit_transform(extended_embeddings)

    # KMeans for cluster-based coloring
    num_clusters = 5  # Adjust as needed
    kmeans = KMeans(n_clusters=num_clusters)
    clusters = kmeans.fit_predict(tsne_results[:-1-top_n])  # Apply KMeans to the sampled embeddings only

    # Plotting
    sns.set(style='whitegrid')
    plt.figure(figsize=(12, 8))

    # Scatter plot for sampled embeddings with cluster coloring
    scatter = plt.scatter(tsne_results[:-1-top_n, 0], tsne_results[:-1-top_n, 1], c=clusters, alpha=0.5, cmap='viridis')

    # Highlight the query
    plt.scatter(tsne_results[-1-top_n, 0], tsne_results[-1-top_n, 1], color='blue', marker='X', s=100)
    plt.annotate(query, (tsne_results[-1-top_n, 0], tsne_results[-1-top_n, 1]), textcoords="offset points", xytext=(0,10), ha='center')

    # Highlight nearest neighbors
    for i, word in enumerate(nearest_neighbors_words):
        nn_index = -top_n + i
        plt.scatter(tsne_results[nn_index, 0], tsne_results[nn_index, 1], color='red', edgecolors='black', s=100)
        plt.annotate(word, (tsne_results[nn_index, 0], tsne_results[nn_index, 1]), textcoords="offset points", xytext=(0,10), ha='center')

    plt.title('t-SNE Visualization with Cluster-Based Coloring, Query, and Neighbors')
    plt.xlabel('t-SNE Component 1')
    plt.ylabel('t-SNE Component 2')
    plt.colorbar(scatter)  # Show color scale for clusters
    plt.show()

# Example usage
# Make sure to replace 'embed_words_batch', 'embeddings_dict', and 'annoy_index' with your actual variables
optimized_plot_embeddings(embeddings_dict, "gpt4", annoy_index, top_n=5)


In [ ]:
import plotly.graph_objects as go
import numpy as np
import seaborn as sns

def plot_embeddings_3d_interactive(embeddings_dict, query, index, top_n=5):
    words, embeddings = zip(*[(word, data['embedding']) for word, data in embeddings_dict.items() if data['embedding'] is not None])
    query_embedding = embeddings_dict.get(query, {'embedding': None})['embedding']
    
    if query_embedding is None:
        query_embedding = embed_words_batch([query])[0]

    extended_embeddings = np.vstack(embeddings + (query_embedding,))

    tsne = TSNE(n_components=3, verbose=1, perplexity=40, n_iter=300)
    tsne_results = tsne.fit_transform(extended_embeddings)

    # Convert Seaborn colors to RGB format acceptable by Plotly
    colors = sns.color_palette("hsv", len(tsne_results))
    colors_rgb = ['rgb' + str(tuple(int(x*255) for x in color)) for color in colors]

    fig = go.Figure()

    # Plot all points with unique colors
    for i, (x, y, z) in enumerate(tsne_results[:-1]):
        fig.add_trace(go.Scatter3d(
            x=[x],
            y=[y],
            z=[z],
            mode='markers',
            marker=dict(size=5, color=colors_rgb[i], opacity=0.5)
        ))

    # Highlight and label the query point
    fig.add_trace(go.Scatter3d(
        x=[tsne_results[-1, 0]],
        y=[tsne_results[-1, 1]],
        z=[tsne_results[-1, 2]],
        mode='markers+text',
        marker=dict(size=8, color='blue'),
        text=[query],
        textposition="bottom center"
    ))

    # Find, highlight, and label nearest neighbors
    indices = index.get_nns_by_vector(query_embedding, top_n)
    for i in indices:
        fig.add_trace(go.Scatter3d(
            x=[tsne_results[i, 0]],
            y=[tsne_results[i, 1]],
            z=[tsne_results[i, 2]],
            mode='markers+text',
            marker=dict(size=8, color='red'),
            text=[words[i]],
            textposition="bottom center"
        ))

    fig.update_layout(
        margin=dict(l=0, r=0, b=0, t=0),
        scene=dict(
            xaxis_title='t-SNE Component 1',
            yaxis_title='t-SNE Component 2',
            zaxis_title='t-SNE Component 3'
        )
    )

    fig.show()


# Assuming embeddings_dict and annoy_index are already created
# Example query word
query_word = "gpt4"  # Replace with your query word

# Plotting the embeddings in 3D with an interactive plot and unique colors
plot_embeddings_3d_interactive(embeddings_dict, query_word, annoy_index, top_n=25)
